# Parameters List
1. k_batch=list(np.arange(1.0, 4.0, 0.5)) # k parameter

2. k_instance=list(np.arange(1.0, 4.0, 0.5)) # This can be set the same as k_batch and hence excluded

3. count_threshold_measure=['median','max'] # alpha parameter

4. logical_parameter=['and','or']



In [1]:
import numpy as np
k_batch=list(np.arange(1.0, 4.5, 0.5))
count_threshold_measure=['median','max']
logical_parameter=['and','or']

In [2]:
import itertools

hyperparameter_space = {
    'k_batch': k_batch,
    'count_threshold_measure':count_threshold_measure,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))


In [3]:
all_combinations

[(1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max', 'and'),
 (3.0, 'max', 'or'),
 (3.5, 'median', 'and'),
 (3.5, 'median', 'or'),
 (3.5, 'max', 'and'),
 (3.5, 'max', 'or'),
 (4.0, 'median', 'and'),
 (4.0, 'median', 'or'),
 (4.0, 'max', 'and'),
 (4.0, 'max', 'or')]

In [4]:
len(all_combinations)

28

### Design the Process flow for grid search

1. Autoencoder is pre-trained , which will be used for all combinations ,this will reduce the processing overhead
2. The validation data needs to be processed once 
3. The iterations will start from threshold computations 
4. The objective is least warnings and detections on normal data set , least detection delay , and more warnnings after detection point
5. We can set the normal data size 20 batches , drifted data size also 20 batches , both combined with drift starting from batch 20
6. We can assign weights to factors in (5) and define a score



In [5]:
total_batches=40
batch_size=32
drift_point=20

WnDs=[i for i in range(0,21)]
print("Warnings in Drift Stream before drift point",WnDs)
DnDs=[i for i in range(0,21)]
print("Detections in Drift Stream before drift point",DnDs)
delay=[ i for i in range(21)]
print("Detection Delay:",delay)
DdDs=[i for i in range(19)]
print("Detections after drift point:",DdDs)
WdDs=[i for i in range(19)]
print("Warnings after drift point:",WdDs)

Warnings in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detection Delay: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Warnings after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [6]:
WnDS=[i for i in range(0,21)]
DnDs=[i for i in range(0,21)]
delay=[ i for i in range(21)]
DdDs=[i for i in range(19)]
WdDs=[i for i in range(19)]

In [7]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    best_value = float('-inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value > best_value:
                            best_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, best_value

# Find the best parameters and the corresponding value
best_parameters, best_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Best parameters:', best_parameters)
print('Value:', best_value)

Best parameters: (0, 0, 0, 18, 18)
Value: 54


In [8]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    worst_value = float('inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value < worst_value:
                            worst_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, worst_value

# Find the best parameters and the corresponding value
best_parameters, worst_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Worst parameters:', best_parameters)
print('Value:', worst_value)

Worst parameters: (20, 20, 20, 0, 0)
Value: -120


# Dataset 

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing


from skmultiflow.data import HyperplaneGenerator
from scipy.stats import norm
import random
from scipy import stats 


import tensorflow
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import anderson

In [10]:
data=pd.read_csv('hyperplane_nondrifted.csv')

In [11]:
data

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,output
0,0.685360,0.953393,0.003948,0.512192,0.812621,0.612526,0.721755,0.291876,0.917774,0.714576,1
1,0.142170,0.373341,0.674134,0.441833,0.434014,0.617767,0.513138,0.650397,0.601039,0.805223,0
2,0.908649,0.319236,0.090459,0.300700,0.113984,0.828681,0.046896,0.626287,0.547586,0.819287,0
3,0.856850,0.351653,0.754648,0.295962,0.883936,0.325512,0.165016,0.392529,0.093460,0.821106,1
4,0.384114,0.944261,0.987625,0.456305,0.826123,0.251374,0.597372,0.902832,0.534558,0.590201,1
...,...,...,...,...,...,...,...,...,...,...,...
39995,0.061714,0.462585,0.943864,0.143777,0.657776,0.325195,0.040887,0.946246,0.313470,0.068780,0
39996,0.073441,0.542813,0.416149,0.585142,0.912097,0.400436,0.582935,0.003902,0.662642,0.179987,0
39997,0.686396,0.536559,0.136439,0.324377,0.606774,0.788541,0.472763,0.443855,0.809508,0.259377,0
39998,0.871673,0.893466,0.046246,0.231087,0.316337,0.445972,0.039865,0.051370,0.502055,0.350849,0


In [12]:
drifted_data=pd.read_csv('hyperplane_drifted.csv')

In [13]:
drifted_data

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,output
0,0.855904,0.753047,0.804241,0.623340,0.608495,0.418715,0.162841,0.105965,0.589660,0.829179,1
1,0.784414,0.408043,0.259734,0.855947,0.919578,0.984292,0.186262,0.663626,0.482852,0.981291,1
2,0.658460,0.033933,0.713858,0.378366,0.906390,0.875112,0.190273,0.245564,0.062429,0.841749,1
3,0.136558,0.231648,0.482025,0.693201,0.245718,0.882909,0.635738,0.522973,0.960885,0.520092,0
4,0.152298,0.665623,0.595665,0.101269,0.544626,0.456066,0.155814,0.648949,0.516301,0.073947,0
...,...,...,...,...,...,...,...,...,...,...,...
2555,0.849341,0.442411,0.238075,0.930455,0.851710,0.134304,0.680624,0.941170,0.695621,0.203228,1
2556,0.333330,0.205006,0.313797,0.816728,0.412498,0.847833,0.961956,0.100099,0.973875,0.380310,0
2557,0.695616,0.735272,0.286571,0.622903,0.656901,0.127594,0.368878,0.212568,0.305806,0.933580,1
2558,0.534210,0.876190,0.020611,0.358940,0.932018,0.659357,0.118251,0.273765,0.597341,0.027397,1


In [14]:
del drifted_data['output']

In [15]:
drifted_data=drifted_data[0:1280]

## 3. Data Preprocessing 
1. Initial 70% data is used for training ( training and validation for AE)
2. Next 20 % data is used as a validation set to compute thresholds
3. Next 10 % data is used as a test stream initilally without drift 
  

In [16]:
def normalize_encode_split(data,label_col,pos_val,neg_val):        
    # Apply Label Encoding 
    # Apply MinMAx Normalization
    # Split into training  , test (validation set 1) and stream ( non-drifted data stream)   
            
    train=data[0:int(len(data)*0.70)]
    test=data[int((0.70*len(data))):int((0.90*len(data)))]
    stream=data[int((0.90*len(data))):len(data)]
    
    train_positives = train[train[label_col] == pos_val]
    train_negatives = train[train[label_col] == neg_val]
    
    
    y_positive=train_positives[label_col] 
    y_negative=train_negatives[label_col]
    X_positive=train_positives.drop([label_col],axis=1)
    X_negative=train_negatives.drop([label_col],axis=1)
    
    
    return train, test, X_positive,X_negative , stream ,y_positive ,y_negative  

In [17]:
train, test, X_positive,X_negative , stream, y_positive ,y_negative =normalize_encode_split(data,'output',1,0)

### Best Autoencoder Hyperparameters will be deternmined for each data set separetely 

In [33]:
def autoencoder(train,epochs,val_set):


    original_dim = 10
    latent_dim = 2

    inputs = tensorflow.keras.Input(shape=(original_dim,))
    z_mean = layers.Dense(latent_dim)(inputs)
    z_log_sigma = layers.Dense(latent_dim)(inputs)

        
    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                                  mean=0., stddev=0.01)
        return z_mean + K.exp(z_log_sigma) * epsilon

    z = layers.Lambda(sampling)([z_mean, z_log_sigma])
    
       
    
    # Create encoder
    encoder = tensorflow.keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
    encoder.summary()

    # Create decoder
    latent_inputs = tensorflow.keras.Input(shape=(latent_dim,), name='z_sampling')
    #x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
    outputs = layers.Dense(original_dim, activation='sigmoid')(latent_inputs)
    decoder = tensorflow.keras.Model(latent_inputs, outputs, name='decoder')
    decoder.summary()

    def compute_vae_loss():
        reconstruction_loss = tensorflow.keras.losses.mean_squared_error(inputs, outputs)
        reconstruction_loss *= original_dim
        kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.1
        vae_loss = K.mean(reconstruction_loss + kl_loss)
        return vae_loss

      
    
    
    # instantiate VAE model
    outputs = decoder(encoder(inputs)[2])
    autoencoder = tensorflow.keras.Model(inputs, outputs, name='vae_mlp')
    
        
     # Define VAE Loss 
    autoencoder_loss=compute_vae_loss()
    autoencoder.add_loss(autoencoder_loss)
    #autoencoder.compile(optimizer='adam')
    autoencoder.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.01))
    
    history=autoencoder.fit(train,train, 
                epochs=epochs,
                batch_size=32,
                validation_data=(val_set, val_set)).history
    
    autoencoder.summary()
    return  autoencoder , history






  def autoencoder(train,epochs,val_set):       
    
        inputs = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
        latent=2
            #H1 = layers.Dense(h1, activation='relu' )(inputs) # Code Layer 1
            
        z_mean = layers.Dense(latent)(inputs)
        z_log_sigma = layers.Dense(latent)(inputs)
            
        def sampling(args):
            z_mean, z_log_sigma = args
            epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent),
                                          mean=0., stddev=0.01)
            return z_mean + K.exp(z_log_sigma) * epsilon            
                  
        z = layers.Lambda(sampling)([z_mean, z_log_sigma])
            
            
        # Create encoder
        encoder = tensorflow.keras.Model(inputs, [z_mean, z_log_sigma, z],name='encoder')
        encoder.summary()
            
        # Create decoder
        latent_inputs = tensorflow.keras.Input(shape=(latent,), name='z_sampling')
        #x = layers.Dense(h1, activation='relu')(latent_inputs)
        outputs = layers.Dense(input, activation='sigmoid')(latent_inputs)
        decoder = tensorflow.keras.Model(latent_inputs, outputs, name='decoder')
        decoder.summary()
            
            
        def compute_vae_loss():
            reconstruction_loss = tensorflow.keras.losses.mean_squared_error( inputs, outputs)
            reconstruction_loss *= input
            kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
            kl_loss = K.sum(kl_loss, axis=-1)
            kl_loss *= -0.1
                
            vae_loss = K.mean(reconstruction_loss + kl_loss)
            return vae_loss
            
        # instantiate VAE model
        outputs = decoder(encoder(inputs)[2])            
        autoencoder = tensorflow.keras.Model(inputs, outputs, name='vae_mlp')
                      
                    
        autoencoder_loss=compute_vae_loss()
        autoencoder.add_loss(autoencoder_loss)
        autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))
           
        print(hyperparameters)
        #print(hyperparameters[0],hyperparameters[1],hyperparameters[2])
            
        history=autoencoder.fit(train,train, 
                    epochs=100,
                    batch_size=32,
                    validation_data=(val_set, val_set)).history  
        autoencoder.summary()    
            
        return  autoencoder , history

In [34]:
def train_encoders(X_Positive,X_Negative, epochs):
    
    X_Positive_train=X_Positive[0:int(len(X_Positive)*0.90)]
    X_Positive_test=X_Positive[int((0.90*len(X_Positive))):len(X_Positive)-1]
    
    X_Negative_train=X_Negative[0:int(len(X_Negative)*0.90)]
    X_Negative_test=X_Negative[int((0.90*len(X_Negative))):len(X_Negative)-1]

    print("Training Autoencoder on Positive Examples ")
    encoder_pos_class, history_positive_class=autoencoder(X_Positive_train,epochs,X_Positive_test)
    print("Training Autoencoder on Negative Examples ")
    encoder_neg_class,history_negative_class=autoencoder(X_Negative_train,epochs,X_Negative_test)
    
    return encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class

In [35]:
encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class=train_encoders(X_positive,X_negative, 100)

Training Autoencoder on Positive Examples 
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 2)            22          input_6[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 2)            22          input_6[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 2)            0           dense_11[0][0]                   
                                                 

394/394 [==============================] - 3s 8ms/step - loss: 0.6089 - val_loss: 0.6132
Epoch 61/100
394/394 [==============================] - 3s 7ms/step - loss: 0.6053 - val_loss: 0.6122
Epoch 62/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6059 - val_loss: 0.6123
Epoch 63/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6070 - val_loss: 0.6118
Epoch 64/100
394/394 [==============================] - 3s 7ms/step - loss: 0.6083 - val_loss: 0.6131
Epoch 65/100
394/394 [==============================] - 3s 7ms/step - loss: 0.6079 - val_loss: 0.6121
Epoch 66/100
394/394 [==============================] - 3s 7ms/step - loss: 0.6069 - val_loss: 0.6137 - ETA: 0s - los
Epoch 67/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6058 - val_loss: 0.6115
Epoch 68/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6090 - val_loss: 0.6108
Epoch 69/100
394/394 [==============================] - 1s 2ms/step - loss: 0.6

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      [(None, 2)]               0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                30        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
394/394 [==============================] - 6s 9ms/step - loss: 0.7313 - val_loss: 0.6250
Epoch 2/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6297 - val_loss: 0.6181
Epoch 3/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6208 - val_loss: 0.6175
Epoch 4/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6199 - val_loss: 0.6140
Epoch 5/100
394/394 [==============================] - 5s 13ms/step - loss: 0.6145 - val_loss: 0.6146
Epoch 6/100
394/394

394/394 [==============================] - 5s 13ms/step - loss: 0.6100 - val_loss: 0.6135
Epoch 76/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6093 - val_loss: 0.6120
Epoch 77/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6112 - val_loss: 0.6127
Epoch 78/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6144 - val_loss: 0.6125
Epoch 79/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6090 - val_loss: 0.6129
Epoch 80/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6120 - val_loss: 0.6136
Epoch 81/100
394/394 [==============================] - 5s 8ms/step - loss: 0.6113 - val_loss: 0.6140
Epoch 82/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6086 - val_loss: 0.6126
Epoch 83/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6100 - val_loss: 0.6157
Epoch 84/100
394/394 [==============================] - 3s 8ms/step - loss: 0.6141 - val_loss:

In [36]:
test

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,output
28000,0.571178,0.497160,0.018756,0.502050,0.557763,0.803582,0.907354,0.147472,0.518609,0.095038,0
28001,0.939351,0.968282,0.897398,0.100867,0.604813,0.217409,0.583915,0.184439,0.359195,0.747322,1
28002,0.568025,0.063038,0.448904,0.528824,0.031582,0.365676,0.230242,0.637741,0.797741,0.579740,0
28003,0.674628,0.832663,0.856485,0.918181,0.055832,0.046599,0.980739,0.832307,0.410864,0.316403,1
28004,0.175722,0.734974,0.752364,0.905388,0.731960,0.474822,0.546828,0.394240,0.824517,0.384729,1
...,...,...,...,...,...,...,...,...,...,...,...
35995,0.148484,0.866953,0.314443,0.207260,0.094772,0.872350,0.213283,0.271287,0.383104,0.363098,0
35996,0.445462,0.547033,0.616315,0.296604,0.238751,0.625008,0.547060,0.625499,0.177021,0.910157,0
35997,0.237011,0.265461,0.253018,0.387041,0.126623,0.996028,0.675586,0.104079,0.722377,0.356402,0
35998,0.222569,0.961094,0.395693,0.889444,0.226125,0.332889,0.762653,0.037542,0.900588,0.050933,0


## Validation Data

In [37]:
def make_batches(test_data): 
        data=np.array(test_data)
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches

In [38]:
def process_validation_data(test,encoder_pos_class,encoder_neg_class,batch_size=32):
   
    # From the test data ,separate positive and negative class data and remove the class column

    test_pos_class=test[test['output']==1]
    test_neg_class=test[test['output']==0]
    del test_pos_class['output']
    del test_neg_class['output']
       
       
    # This function make batches of the data stream
     
    def make_batches(test_data):
        data=np.array(test_data)
        batch_size=32
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches
    #
    
    batches_pos=make_batches(test_pos_class)
    batches_neg=make_batches(test_neg_class)
    
    # This function computes reconconstruction error for each instance as well as average reco.error of a batch passed to it. 
    def compute_instance_loss_batch_loss(batch,batch_size,encoder):
        mse_list=[]
        mse_sum=0  
        for i in range(0,batch.shape[0]):
            ROW = np.array([batch[i]])
            pred= encoder.predict(ROW) 
            mse = np.mean(np.power(batch[i] - pred, 2))
            mse_list.append(mse)
            mse_sum+=mse
        avg_mse=mse_sum/batch_size
        return mse_list,avg_mse   

            
    # This function computes recon.errr of all the batches .  
    def process_all_batches(batches,encoder):
        batch_avg_mse=[]
        batch_mse_values={}
        for b in batches:
            print(b,end=" ")
            mse_list,average_mse=compute_instance_loss_batch_loss(batches[b],batch_size,encoder)
            batch_avg_mse.append(average_mse)
            batch_mse_values[b]=mse_list
        return batch_avg_mse,batch_mse_values

    batch_avg_mse_pos,batch_mse_values_pos=process_all_batches(batches_pos,encoder_pos_class)
    batch_avg_mse_neg,batch_mse_values_neg=process_all_batches(batches_neg,encoder_neg_class)
       
    
    
    return  batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg
    
    
    
   
    

In [39]:
batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg=process_validation_data(test,encoder_pos_class,encoder_neg_class)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 

In [40]:
k_parameter=list(np.arange(1.0, 4.5, 0.5))
alpha_parameter=['median','max']
logical_parameter=['and','or']
import itertools

hyperparameter_space = {
    'k': k_parameter,
    'count_threshold_measure':alpha_parameter,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))
all_combinations

[(1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max', 'and'),
 (3.0, 'max', 'or'),
 (3.5, 'median', 'and'),
 (3.5, 'median', 'or'),
 (3.5, 'max', 'and'),
 (3.5, 'max', 'or'),
 (4.0, 'median', 'and'),
 (4.0, 'median', 'or'),
 (4.0, 'max', 'and'),
 (4.0, 'max', 'or')]

In [41]:
len(all_combinations)

28

### AEDDM Framwework

In [42]:
def detect_at_batch_level(test_batch,b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list):
    
    # Layer 1 Variables 
    mse_list_layer1=[]     # Holds the recon loss values predicted by Layer 1 Autoencoder for a single batch
    exceed_count_layer1=0  # How many instances exceed layer one instance level threshold
    mse_sum=0              # sum of recon.error values from layer 1 AE for this batch
    
    
    
    # Layer 2 Variables 
    mse_list_layer2=[]      # Holds the recon.error values predicted by Layer 2 Autoencoder for a single batch
    exceed_count_layer2=-1  # If a batch is not passed to the layer2 AE , then exceed count value will be -1 . This is symbolic.
    mse_sum_layer2=0        # sum of recon.error values from layer 2 AE for this batch
    
        
    layer1_excede_list=[] # Holds the batch numbers of bathces exceeding layer1 thresholds
    #all_excede_list=[]    # Hold the batch number of batches exceeding both Layer-1 and Layer-2 AE Thresholds 
    layer_one_instance_exceed_list=[] # Holds the indices of instances exceeding layer 1 instance threshold
    layer_two_instance_exceed_list=[] # Holds the indices of instances exceeding layer 2 instance thresholds
     
      
    # Determine Layer 1 and Layer AE and their associated thresholds
    layer_one_batch_thres= batch_thres_pos if batch_thres_pos<batch_thres_neg  else batch_thres_neg
    layer_two_batch_thres= batch_thres_pos if batch_thres_pos>batch_thres_neg  else batch_thres_neg
    layer_one_encoder= encoder_pos_class if batch_thres_pos<batch_thres_neg else encoder_neg_class
    layer_two_encoder= encoder_pos_class if batch_thres_pos>batch_thres_neg else encoder_neg_class
    
    layer_one_count_threshold=count_thresh_pos if batch_thres_pos<batch_thres_neg  else count_thres_neg
    layer_two_count_threshold=count_thresh_pos if batch_thres_pos>batch_thres_neg  else count_thres_neg
    
    layer1_ins_thresh=instance_thresh_pos if  batch_thres_pos<batch_thres_neg  else instance_thresh_neg
    layer2_ins_thresh=instance_thresh_pos if  batch_thres_pos>batch_thres_neg  else instance_thresh_neg
    
    # Pass each instance of a batch to Layer 1 AE. Compute Batch MSE and Number of Instances excceding Layer 1 Instance Thresh       
    avg_mse_layer1=0
    for i in range(0,test_batch.shape[0]):
        ROW = np.array([test_batch[i]])
        pred= layer_one_encoder.predict(ROW) 
        mse = np.mean(np.power(test_batch[i] - pred, 2))
        mse_list_layer1.append(mse)
        if mse>layer1_ins_thresh:
            exceed_count_layer1+=1
            layer_one_instance_exceed_list.append(i)
        mse_sum+=mse    
    avg_mse_layer1=(mse_sum)/len(test_batch)
    avg_mse_layer2=0 
    # Check if This batch exceeds both Layer 1 batch and count thresholds
    if ((avg_mse_layer1>layer_one_batch_thres) and ( exceed_count_layer1 >layer_one_count_threshold) ):
        layer1_excede_list.append(b)# Keep track of batches exceesding layer 1 thresholds
        exceed_count_layer2=0
        # Pass each instance of this batch to Layer 2 AE. Compute Batch MSE and Number of Instances excceding Layer 2 Instance Thresh
        for i in range(0,test_batch.shape[0]):
            ROW = np.array([test_batch[i]])
            pred= layer_two_encoder.predict(ROW) 
            mse = np.mean(np.power(test_batch[i] - pred, 2))
            mse_list_layer2.append(mse)
            if mse>layer2_ins_thresh:
                exceed_count_layer2+=1
                layer_two_instance_exceed_list.append(i)
            mse_sum_layer2+=mse    
        avg_mse_layer2=(mse_sum_layer2)/len(test_batch)
        
        if  (avg_mse_layer2 > layer_two_batch_thres) and (exceed_count_layer2>layer_two_count_threshold):
            all_excede_list.append(b)# Keep track of batches exceesding layer 2 thresholds
          
    return  all_excede_list,mse_list_layer1 , exceed_count_layer1 ,avg_mse_layer2,exceed_count_layer2,len(layer_two_instance_exceed_list),layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list,layer_two_instance_exceed_list,avg_mse_layer1 

In [43]:
def detect_stream_drift(batches,encoder_pos_class,encoder_neg_class,batch_thres_pos,batch_thres_neg, count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg,logical):
    exceed_count_layer2_instance_thresh={} # Holds Number of instances exceeding thresholds of Both Layer 1 and layer 2 AE.
    
    mse_dict_L1={}         # Holds batchwise recon.error values from Layer 1 AE
    exceed_count_L1={}     # Batchwise number of Instances exceeding layer 1 count threshold
    layer_one_instance_exceed_list={} # batch wise list of instances ( indices ) exceeding Layer 1 count threshold
    avg_mse_l1={}
    
    mse_dict_L2={}        # Holds batchwise recon. error  values from Layer 2 AE
    exceed_count_L2={}    # Batchwise number of Instances exceeding layer 2 count threshold
    layer_two_instance_exceed_list={} # For each batch maintains the indices where recon.error exceeds layer 2 threshold
    avg_mse_l2={}         #
    all_excede_list=[]
    n=0
    for b in batches:
        print("\n\n")
        print("**************")
        print('\nBatch Number : {}'.format(b))
        all_excede_list,mse_dict_L1[b],exceed_count_L1[b],avg_mse_l2[b],exceed_count_L2[b],exceed_count_layer2_instance_thresh[b],layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list[b],layer_two_instance_exceed_list[b],avg_mse_l1[b]= detect_at_batch_level(batches[b],b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list)
             
        
        print('\nData Points Exceeding Layer 1 Encoder Instance Threshold : {} '.format(layer_one_instance_exceed_list[b]))
        print('\nData Points Exceeding Layer 2 Encoder Instance Threshold: {}'.format(layer_two_instance_exceed_list[b])) 
        print('\nNumber of Data Points Exceeding Layer 2 Encoder Instance thresholds: {}'.format(exceed_count_layer2_instance_thresh[b]))
        
    mse_list_layer1=return_list_of_dict_values(avg_mse_l1)
    exceed_count_list_layer1=return_list_of_dict_values(exceed_count_L1)
    
    
    ## Detect Drift at Batch Level 
    print ("\n Drift Detection at Batch Level\n")
    exceed_list=return_list_of_dict_values(exceed_count_L2)
    mse_list=return_list_of_dict_values(avg_mse_l2)
    score,col_values=detect_drift(mse_list,exceed_list,layer_two_batch_thres,layer_two_count_threshold,logical)
    
    
    return score,col_values,all_excede_list,exceed_count_layer2_instance_thresh ,exceed_count_L2,avg_mse_l2 ,mse_list_layer1,exceed_count_list_layer1




In [44]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=20
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    score=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={score}")
    
    col_values=[WnDs,DnDs,delay,WdDs,DdDs,score]
    
    return score,col_values


In [45]:
results_df_p=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [46]:
# This function computes how many instances in a batch exceed instance threshold
def threshold_exceed_count(batch_mse_values, thr):
    exceed_count={}
    for key in batch_mse_values.keys():
        count=0
        list=batch_mse_values[key]
        for a in range(0,len(list)):
            if list[a]>thr:
                count+=1
        exceed_count[key]=count 
    values = exceed_count.values()
    
    return exceed_count

def compute_count_threshold(exceed_count_dict,alpha):
    exceed_list=[]
    for key in exceed_count_dict.keys():
        exceed_list.append(exceed_count_dict[key]) 
    #print(" from with in count threshold")
    #print(alpha)
    method = getattr(np,alpha)  # this enables np.median() or np.max() based on alpha value
    count_threshold=method(exceed_list)    
    return count_threshold
   
def compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse,batch_avg_mse_values,k,alpha,N=100):
        # Computes Instance Threshold
        zscore_list=[]
        for i in range(0,N):
            value_list=batch_avg_mse_values[i]

            mean=np.mean(value_list)
            sigma=np.std(value_list)
            thres_zscore=mean+k*sigma
            zscore_list.append(thres_zscore)
        # Computes batch threshold
        mean_batch=np.mean(batch_avg_mse)
        sigma_batch=np.std(batch_avg_mse)    
        batch_thresh=mean_batch+k*sigma_batch
        
        method = getattr(np,alpha)
        instance_thresh = method(zscore_list)
        
        return  instance_thresh, zscore_list , batch_thresh


    
# Make batches of the normal data stream     
s=stream.copy()
del s['output']
batches_n=make_batches(s)
#print(batches_n)
# For testing purpose choose 20 batches only
batches_n=dict(itertools.islice(batches_n.items(), 20))
    
# Drifted Data Stream
drifted=drifted_data[0:1280]
batches_d=make_batches(drifted)    
    
    
    
def return_list_of_dict_values(d):
    values_list=[]
    for key in d.keys():
        values_list.append(d[key])
    return values_list
    

    
    
def pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg):
    # Define the parametric space
    k_parameter=list(np.arange(0.5, 4.5, 0.5))
    alpha_parameter=['median','max']
    logical_parameter=['and','or']
    results={}
   
    global results_df_p
    row=0
    
    
    for k in k_parameter:
        for alpha in alpha_parameter:
            for logical in logical_parameter:
                print("********************************")
                
                print(k,alpha,logical)
                values=[k, alpha, logical]
                parameters=(k,alpha,logical)
                instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
                instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
                #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
                # Counts the MSE values exceeding threshold in each batch 
                exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
                exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
                count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
                count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
                print("Thresholds for Positive AE")
                #print(alpha)
                print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
                print("Thresholds for Nagative AE")
                print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
                # AEDDM Starts Here 
                
                thres_zscore_batch_pos=batch_thresh_pos
                thres_zscore_batch_neg=batch_thresh_neg
                             
                
                #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
                    #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
                print(" Drifted Data Processing Starts Here")
                
                score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    
        
                results[parameters]=score
                values.extend(col_values)
                results_df_p = results_df_p.append(pd.Series(values, index=results_df_p.columns), ignore_index=True)
                print(results_df_p)
    
    return results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d

In [47]:
results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d=pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg)

********************************
0.5 median and
Thresholds for Positive AE
Batch Thresold: 0.06308798787312297 , Count Threshold: 10.0
Thresholds for Nagative AE
Batch Thresold: 0.06329431629938527 , Count Threshold: 10.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 6, 7, 9, 12, 13, 16, 19, 20, 21, 22, 23, 25, 26, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 5, 16, 17, 18, 20, 22, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 9



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 4, 5, 7, 8, 10, 12, 19, 20, 21, 23, 24, 25, 26, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 6, 7, 10, 12, 13, 16, 21, 23, 24, 25, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 2

Data Points Exceeding Laye


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 4, 5, 8, 10, 13, 16, 22, 23, 25, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 4, 6, 9, 10, 12, 16, 22, 24, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 12



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 4, 6, 7, 8, 9, 12, 13, 16, 18, 19, 22, 23, 24, 25, 26, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 3, 4, 5, 7, 9, 11, 14, 18, 19, 22, 23, 24, 25, 26, 28, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 19



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 4, 8, 9, 11, 14, 15, 17, 18, 22, 26, 27, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 4, 5, 6, 11, 12, 16, 17, 18, 22, 23, 26, 27, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Inst


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 6, 9, 12, 13, 16, 19, 20, 21, 22, 23, 25, 26, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 5, 16, 17, 18, 20, 22, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 9



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 4, 5, 7, 8, 10, 12, 19, 20, 21, 23, 24, 25, 26, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 6, 7, 10, 12, 13, 16, 21, 23, 24, 25, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 5, 7, 9, 10, 11, 13, 14, 17, 20, 21, 22, 24, 25, 26, 27, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 4, 5, 8, 9, 14, 17, 18, 20, 21, 23, 26, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14





Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 4, 6, 7, 8, 9, 12, 13, 16, 18, 19, 22, 23, 24, 25, 26, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 3, 4, 5, 7, 9, 11, 14, 18, 19, 22, 23, 24, 25, 26, 28, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 19



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 4, 8, 9, 11, 14, 15, 17, 18, 22, 26, 27, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 4, 5, 6, 11, 12, 16, 17, 18, 22, 23, 26, 27, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 17



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 7, 8, 9, 10, 11, 12, 20, 21, 23, 25, 26, 27, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 3, 4, 8, 10, 12, 19, 22, 25, 26, 27, 28, 29, 30]

Number of Data Points Exceeding Layer


Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 6, 9, 13, 16, 19, 20, 21, 22, 25, 27, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 18, 20, 22, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 7, 10, 12, 19, 21, 23, 25, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 5, 9, 11, 13, 20, 22, 25, 26, 27, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 4, 5, 8, 14, 20, 21, 23]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 8



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 17, 19, 21, 22, 23, 30] 

Data Points Exceeding Layer

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 22

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 8

Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 16, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 9

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 10

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Thres


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 



Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 39

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 22

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : 

In [48]:
results_df_p

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.5,median,and,16,16,5,5,5,72
1,0.5,median,or,13,13,3,7,7,93
2,0.5,max,and,0,0,20,0,0,60
3,0.5,max,or,3,3,5,2,2,102
4,1.0,median,and,16,16,5,4,4,69
5,1.0,median,or,20,20,3,9,9,78
6,1.0,max,and,0,0,20,0,0,60
7,1.0,max,or,5,5,5,2,2,96
8,1.5,median,and,11,11,5,2,2,78
9,1.5,median,or,20,20,3,6,6,69


In [49]:
results_df=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [50]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Categorical, Real


def aeddm_function(x):
    global results_df
    row=0
    k,alpha,logical=x
    values=[k, alpha, logical]
    #results_df.iloc[row :3] = values
        
    print("***************************************************")
    print(k,alpha,logical)
    parameters=(k,alpha,logical)
    instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
    instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
    #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
    # Counts the MSE values exceeding threshold in each batch 
    exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
    exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
    count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
    count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
    print("Thresholds for Positive AE")
    #print(alpha)
    print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
    print("Thresholds for Nagative AE")
    print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
    # AEDDM Starts Here 
                
    thres_zscore_batch_pos=batch_thresh_pos
    thres_zscore_batch_neg=batch_thresh_neg
                             
                
    #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
        #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
             #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    print(" Drifted Data Processing Starts Here")
                
    score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    #value+=100
    print(col_values)
    print(values)
    values.extend(col_values)
    print(values)
    
    results_df = results_df.append(pd.Series(values, index=results_df.columns), ignore_index=True)
    print(results_df)
    
    values=[]
    col_values=[]
    print(k,alpha,logical)
    return score 



In [51]:
# Define the search space
row=0
search_space = [
    Real(0.5, 4),  # Numerical parameter k with range 1 to 4
    Categorical(['median', 'max']),  # Categorical parameter alpha
    Categorical(['and', 'or'])  # Categorical parameter logical
]

# Perform the Bayesian optimization (maximization)
result = gp_minimize(
    #aeddm_function,
    lambda x: -aeddm_function(x),  # Negate the function to maximize
    dimensions=search_space,
    n_calls=20,  # Number of iterations
    acq_func='EI',  # Expected Improvement for maximization
    random_state=1  # Set a random seed for reproducibility
)

# Get the best parameters found
best_params = result.x
print("Best parameters:", best_params)

***************************************************
3.9901468382860408 median and
Thresholds for Positive AE
Batch Thresold: 0.07730307285460972 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 0.0771473126720116 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [10, 21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [21, 23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [17] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshold


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 30

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 12, 15, 17, 26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 5, 6, 10, 14, 15, 16, 17, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 9



**************

Batch Number : 17

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 10, 16, 22, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [4, 15, 21, 25, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 18

Data Points Exceeding Layer 1 Encoder Instance Threshold : [24, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 16, 19, 23, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 19

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 5, 14, 19, 21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 


...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 12, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 5, 10, 24, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 16

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 15, 26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 17

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 10] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 18

Data Points Exceeding Layer 1 Encoder Instance Threshold : [24, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [23]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 19

Data Points 

***************************************************
0.5003920772951769 max or
Thresholds for Positive AE
Batch Thresold: 0.06308958477214427 , Count Threshold: 11
Thresholds for Nagative AE
Batch Thresold: 0.06329587252197404 , Count Threshold: 10
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 6, 9, 13, 16, 19, 20, 21, 22, 25, 27, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 18, 20, 22, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 7, 10, 12, 19, 21, 23, 25, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 5, 9, 11, 13, 20, 22, 25, 26, 27, 29, 30, 31]


Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 6, 16, 17, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 5, 8, 9, 17, 23, 24, 25, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 30

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 4, 7, 8, 11, 12, 16, 22, 23, 26, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 4, 7, 8, 10, 15, 21, 22, 24, 26, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 13



**************

Batch Number : 31

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 5, 6, 14, 15, 17, 18, 23, 25, 26, 30] 

Data Points Exceeding Layer 2 Encoder In


Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 15, 16, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 10

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 5, 8, 9, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 13, 17, 22, 23, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 22, 23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 12

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 13, 16, 18, 23, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [8, 9, 13, 18, 21, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance 

Best parameters: [1.4556749097586161, 'max', 'or']


In [52]:
max_value = result.fun  # Access the maximum function value
print("Maximum value:", max_value)
print("Best parameters:", result.x)

Maximum value: -111
Best parameters: [1.4556749097586161, 'max', 'or']


In [53]:
df_EI=results_df.copy()

In [54]:
df_EI

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,3.990147,median,and,0,0,20,0,0,60
1,3.996642,max,and,0,0,20,0,0,60
2,1.857688,median,or,11,11,5,6,6,90
3,3.462088,max,or,0,0,20,0,0,60
4,2.052085,max,or,0,0,20,0,0,60
5,3.698867,max,and,0,0,20,0,0,60
6,3.786947,median,or,0,0,20,0,0,60
7,3.309651,max,or,0,0,20,0,0,60
8,3.527571,median,or,0,0,20,0,0,60
9,1.455675,max,or,0,0,5,2,2,111


In [55]:
results_df=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [56]:
# Define the search space
search_space = [
    Real(0.5, 4),  # Numerical parameter k with range 1 to 4
    Categorical(['median', 'max']),  # Categorical parameter alpha
    Categorical(['and', 'or'])  # Categorical parameter logical
]

# Perform the Bayesian optimization (maximization)
result = gp_minimize(
    
    lambda x: -aeddm_function(x),  # Negate the function to maximize
    dimensions=search_space,
    n_calls=20,  # Number of iterations
    acq_func='PI',  # Expected Improvement for maximization
    random_state=1  # Set a random seed for reproducibility
)

# Get the best parameters found
best_params = result.x
print("Best parameters:", best_params)

***************************************************
3.9901468382860408 median and
Thresholds for Positive AE
Batch Thresold: 0.07730307285460972 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 0.0771473126720116 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [10, 21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [21, 23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [17] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshold


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 30

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 12, 15, 17, 26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 5, 6, 10, 14, 15, 16, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 8



**************

Batch Number : 17

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 10, 16, 22, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [4, 15, 21, 25, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 18

Data Points Exceeding Layer 1 Encoder Instance Threshold : [24, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 16, 19, 23, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 19

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 5, 14, 19, 21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Ins


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 


...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 4, 10, 17, 22] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 8]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 15

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 8, 12, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 5, 8, 10, 24, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 16

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 15, 26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 17

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 10] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch N

***************************************************
1.459297206239113 max or
Thresholds for Positive AE
Batch Thresold: 0.06699512758415258 , Count Threshold: 3
Thresholds for Nagative AE
Batch Thresold: 0.06710193280781478 , Count Threshold: 3
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [20, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 12, 21, 23, 26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [10, 12, 21, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 9, 13, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [14, 21, 23]

Number of Data Points Ex


Data Points Exceeding Layer 1 Encoder Instance Threshold : [6, 8, 16, 17, 24, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [24]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 33

Data Points Exceeding Layer 1 Encoder Instance Threshold : [6, 24, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 34

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 11, 15, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [21]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [19, 24, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Dat


Data Points Exceeding Layer 1 Encoder Instance Threshold : [13, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 14

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 4, 10, 17, 22] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 8]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 15

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 8, 12, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 5, 8, 10, 24, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 16

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 15, 26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch 

Best parameters: [1.4556749097586161, 'max', 'or']


In [57]:
max_value = result.fun  # Access the maximum function value
print("Maximum value:", max_value)
print("Best parameters:", result.x)

Maximum value: -111
Best parameters: [1.4556749097586161, 'max', 'or']


In [58]:
results_df

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,3.990147,median,and,0,0,20,0,0,60
1,3.996642,max,and,0,0,20,0,0,60
2,1.857688,median,or,11,11,5,6,6,90
3,3.462088,max,or,0,0,20,0,0,60
4,2.052085,max,or,0,0,20,0,0,60
5,3.698867,max,and,0,0,20,0,0,60
6,3.786947,median,or,0,0,20,0,0,60
7,3.309651,max,or,0,0,20,0,0,60
8,3.527571,median,or,0,0,20,0,0,60
9,1.455675,max,or,0,0,5,2,2,111


In [59]:
Thresh=0.212964
count_thresh=0
logical='and'

In [60]:
drift_point=20
WnDs= 0 
WdDs=0 
DnDs=0          
delay=None     
DdDs=0   

In [63]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift2(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=20
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    value=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={value}")
    
    

In [64]:
detect_drift2(avg_mse_l2_list_d,exceed_count_L2_countThresh_d, Thresh,count_thresh,logical)

Number of Drifted Batches0
All Drifted Batches:  []
Drifted Batches before Drift Point:  []
Drifted Batches after Drift Point:  []
WnDs=0 ,WdDs=0 , DnDs=0,DdDs=0 , delay= 20 , value=60
